In [8]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [5]:
cancer = load_breast_cancer()

In [7]:
cancer

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [9]:
x_train,x_test,y_train,y_test = train_test_split(cancer.data,cancer.target,test_size=0.2)

In [10]:
pd.DataFrame(x_train).shape

(455, 30)

In [11]:
pd.DataFrame(y_train).shape

(455, 1)

In [12]:
pd.DataFrame(x_test).shape

(114, 30)

In [13]:
from sklearn.model_selection import StratifiedKFold
StratifiedKFold(n_splits=10,random_state=1)

StratifiedKFold(n_splits=10, random_state=1, shuffle=False)

In [14]:
from sklearn.model_selection import StratifiedKFold
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((0,1),float)
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
        test_pred=np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred

In [15]:
from sklearn import tree
model1 = tree.DecisionTreeClassifier(random_state=1)
test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=pd.DataFrame(x_train),test=pd.DataFrame(x_test),y=pd.Series(y_train))
train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [16]:
test_pred1

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,0.0
5,0.0
6,0.0
7,1.0
8,1.0
9,1.0


In [17]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier()
test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=pd.DataFrame(x_train),test=pd.DataFrame(x_test),y=pd.Series(y_train))
train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [18]:
voted_test_pred1 = pd.DataFrame(list(map(lambda x: 1 if x>=5 else 0,test_pred1[0].values.reshape(10,y_test.shape[0]).sum(axis=0))))
voted_test_pred2 = pd.DataFrame(list(map(lambda x: 1 if x>=5 else 0,test_pred2[0].values.reshape(10,y_test.shape[0]).sum(axis=0))))

In [19]:
from sklearn.linear_model import LogisticRegression
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([voted_test_pred1, voted_test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

E:\Tools\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9122807017543859

In [20]:
model1.score(x_test,y_test),model2.score(x_test,y_test)

(0.9298245614035088, 0.9298245614035088)